In [2]:
import cv2
import numpy as np
import mediapipe as mp

# Initialize MediaPipe Face Detection and Face Mesh
mp_face_detection = mp.solutions.face_detection
mp_face_mesh = mp.solutions.face_mesh
face_detection = mp_face_detection.FaceDetection(min_detection_confidence=0.7)
face_mesh = mp_face_mesh.FaceMesh(max_num_faces=1, min_detection_confidence=0.7)

def detect_faces(image):
    """Detect faces in an image using MediaPipe."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_detection.process(image_rgb)
    if results.detections:
        faces = [detection.location_data.relative_bounding_box for detection in results.detections]
        return faces
    return []

def get_face_landmarks(image, face_box):
    """Detect facial landmarks in a cropped face image."""
    h, w, _ = image.shape
    ymin = int(face_box.ymin * h)
    xmin = int(face_box.xmin * w)
    ymax = int((face_box.ymin + face_box.height) * h)
    xmax = int((face_box.xmin + face_box.width) * w)
    
    face_crop = image[ymin:ymax, xmin:xmax]
    face_crop_rgb = cv2.cvtColor(face_crop, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(face_crop_rgb)
    
    landmarks = []
    if results.multi_face_landmarks:
        for landmarks_list in results.multi_face_landmarks:
            landmarks = [(lm.x * face_crop.shape[1], lm.y * face_crop.shape[0]) for lm in landmarks_list.landmark]
    return landmarks, face_crop

def align_faces_using_homography(src_landmarks, tgt_landmarks, src_face, tgt_face):
    """Align the target face to the source face using homography."""
    src_points = np.array(src_landmarks, dtype=np.float32)
    tgt_points = np.array(tgt_landmarks, dtype=np.float32)

    # Find homography matrix
    H, status = cv2.findHomography(tgt_points, src_points)
    aligned_face = cv2.warpPerspective(tgt_face, H, (src_face.shape[1], src_face.shape[0]), flags=cv2.INTER_LINEAR)
    
    return aligned_face

def compute_pixel_difference(image1, image2):
    """Compute the pixel-wise difference between two images."""
    image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
    difference = cv2.absdiff(image1_gray, image2_gray)
    _, thresholded_diff = cv2.threshold(difference, 25, 255, cv2.THRESH_BINARY)
    return difference

def main():
    # Load images
    image_before = cv2.imread('data/after/1.png')
    image_after = cv2.imread('data/before/1.png')

    # Detect faces
    faces_before = detect_faces(image_before)
    faces_after = detect_faces(image_after)

    if faces_before and faces_after:
        # Assume we take the first detected face for alignment
        face_box_before = faces_before[0]
        face_box_after = faces_after[0]
        
        # Get landmarks and crop faces
        landmarks_before, face_crop_before = get_face_landmarks(image_before, face_box_before)
        landmarks_after, face_crop_after = get_face_landmarks(image_after, face_box_after)
        
        if landmarks_before and landmarks_after:
            # Align faces using homography
            aligned_after_face = align_faces_using_homography(landmarks_before, landmarks_after, face_crop_before, face_crop_after)
            
            # Scale faces to the same size
            size = (face_crop_before.shape[1], face_crop_before.shape[0])
            aligned_after_face = cv2.resize(aligned_after_face, size)
            
            # Compute pixel difference
            pixel_difference = compute_pixel_difference(face_crop_before, aligned_after_face)
            
            # Display results
            cv2.imshow('Face Before Makeup', face_crop_before)
            cv2.imshow('Aligned After Makeup Face', aligned_after_face)
            cv2.imshow('Pixel Difference', pixel_difference)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        else:
            print("Landmarks could not be detected.")
    else:
        print("Faces could not be detected.")

if __name__ == "__main__":
    main()
